# LISAs BASED CSV GENERATOR

## GFM GENERATOR

In [ ]:
import os
import pandas as pd

# Load the main data (CGM data with timestamps and glucose values)
df = pd.read_csv('')


# Initialize a list to store results
results = []

for i in range(1, 41):
    GFM = 0
    count = 0


    # Iterate over the patient data
    for row_index in range(len(df)):
        count += 1
        if count > 288:
            # Define the mean CGM for 24 hours
            last_24h_cgm = df['glucose'].iloc[row_index-288:row_index].dropna()
            CGM_L = last_24h_cgm.mean()

            # Define the mean CGM for 1 hour
            last_h_cgm = df['glucose'].iloc[row_index-12:row_index].dropna()
            CGM_S = last_h_cgm.mean()

            # Define AUC and GFM
            if CGM_S > CGM_L:
                AUC = (CGM_S - CGM_L) / 5 # The article doesn't make sense numberwise
                GFM += AUC
            else:
                GFM = 0
        else:
            GFM = 0

        # Append the result with timestamp and GFM
        results.append({'timestamp': df['timestamp'].iloc[row_index], 'GFM': GFM})

# Create a DataFrame from the results
lisa_df = pd.DataFrame(results)

# Save the DataFrame as a .csv file
output_file = './Visualizer_all_boluses/lisa_gfm.csv'
lisa_df.to_csv(output_file, index=False)

print(f'Results saved to {output_file}')


## IFM GENERATOR

We need to proceed with the calculation of IFM, for that, we first need to calculate the PIE, which is the second order filter version of the first order filter IOB. This code runs on raw insulin intake, not IOB. Along with the insulin intake, there's a couple matrixes that ought to do the job.

If you have the IOB ready or have calculated it another way, you ought to apply a first order filter on the IOB timestamps to calculate the PIE.

In [ ]:
import numpy as np
import pandas as pd

# Create an empty DataFrame to collect results
result_df = pd.DataFrame()


df1 = pd.read_csv('') # Load the insulin data for the patient

# Initialize state variables
x1_k = 0
x2_k = 0
A = np.array([[0.98, 0.02],
                [0, 0.98]])
B = np.array([0, 1])

PIE_values = []
Total_Insulin = []
counts = 0
# Calculate Total_Insulin
for row_index in range(len(df1)):
    if df1.iloc[row_index]['LastBolus'] < 180: #'LastBolus' is time paassed since last bolus in seconds   
        Total_Insulin.append(df1.iloc[row_index]['Basal']/200 + df1.iloc[row_index]['Bolus']/100) #Basal needs to be in U/3 min ; Bolus in 100U
    else:
        Total_Insulin.append(df1.iloc[row_index]['Basal']/200)
        
# Loop through the insulin data and update state variables
for insulin_k in Total_Insulin:
    x1_k_next, x2_k_next = np.dot(A, [x1_k, x2_k]) + B * insulin_k
            
    # Update state variables for the next iteration
    x1_k = x1_k_next
    x2_k = x2_k_next

    
    # Calculate PIE_k
    PIE_k = x1_k  # Since PIE_k = [1 0] * [x1_k, x2_k]
            
    # Store the PIE value
    PIE_values.append(PIE_k)
            
# Add the PIE values to the DataFrame
df1['PIE_k'] = PIE_values


PIE_24h_list = []
PIE_1h_list = []
IFM_list = []

# Calculate PIE_24h, PIE_1h, and IFM
for row in range(len(df1)):
    if row >= 480:
        PIE_24h = df1['PIE_mean'].iloc[row-480:row].mean() # change for PIE_mean
        PIE_1h = df1['PIE_mean'].iloc[row-20:row].mean()
        IFM = (PIE_1h - PIE_24h) / PIE_24h
    else:
        PIE_24h = 0
        PIE_1h = 0
        IFM = 0
        
    PIE_24h_list.append(PIE_24h)
    PIE_1h_list.append(PIE_1h)
    IFM_list.append(IFM)

# Assign lists to DataFrame columns
df1['PIE_24h'] = PIE_24h_list
df1['PIE_1h'] = PIE_1h_list
df1['IFM'] = IFM_list

# Select only the required columns
df2 = df1[['timestamp', 'IFM', 'Basal']] 

## FINAL DATASET LISAs  

In [ ]:
# Append the results to the result DataFrame
result_df = pd.concat([result_df, df2], ignore_index=True)


# Save the final result to IFM.csv
result_df.to_csv('./Visualizer_all_boluses/lisa_ifm.csv', index=False)

# Load the CGM data
gfm_df = pd.read_csv('./Visualizer_all_boluses/lisa_gfm.csv')

# Merge them with the proper timestamps
result_df['timestamp'] = pd.to_datetime(result_df['timestamp'])
gfm_df['timestamp'] = pd.to_datetime(gfm_df['timestamp'])


merged_df = pd.merge_asof(result_df.sort_values('timestamp'), 
                          gfm_df.sort_values('timestamp'), 
                          on='timestamp', 
                          direction='nearest')

merged_df = merged_df.sort_values(['timestamp'])

# LISAs article main findings (IFM > 0.4 & GFM > 100 & Basal > 1.5 (10U/hr))
for _, row in merged_df.iterrows():
    if not merged_df.empty and merged_df['IFM'].gt(0.4).all() and merged_df['GFM'].gt(100).all() and merged_df['Basal'].gt(1.5).all():
        lisas = 1 # This is the condition for LISAs alarm to be activated
    else:
        lisas = 0

    row.append(lisas)

merged_df.to_csv('./LISAs.csv', index=False)